In [5]:
import pandas as pd

In [6]:
def calculate_alpha(row):
    lot_size = 40  # Default lot size for FINNIFTY
    
    if 'BANKNIFTY' in row['Portfolio']:
        lot_size = 25
    elif 'FINNIFTY' in row['Portfolio']:
        lot_size = 40
    elif 'MIDCPNIFTY' in row['Portfolio']:
        lot_size = 15
    elif 'NIFTY' in row['Portfolio']:
        lot_size = 25
    
    alpha = (row['TRDPARITY'] - row['ParityAsked']) * row['QTY'] * lot_size
    return alpha

In [8]:

# Initialize a list to store parsed data
data = []

# Read the file
with open('Trade logs//4L_Trades.txt', 'r') as file:
    lines = file.readlines()
    
    # Loop through each line
    for line in lines:
        if '|' in line:  # Skip lines that do not contain '|'
            # Split the line by the '|' separator
            fields = line.split('|')
            
            # Remove leading and trailing whitespace from each field
            fields = [field.strip() for field in fields]

            # Append the fields to the data list
            data.append(fields)

# Create a DataFrame from the data list
df = pd.DataFrame(data, columns=[
    'Timestamp', 'Description', 'TRDPARITY', 'QTY', 'ParityWas', 
    'OpnCls', 'OrdNumL1', 'OrdNumL2', 'OrdNumL3', 'OrdNumL4', 'M2M'
])

# Define a function to clean the values
def clean_values(series, col_name):
    return series.str.replace(f'{col_name}:', '', regex=False)

# Apply the function to each column except Timestamp and Description
for col in df.columns:
    if col not in ['Timestamp', 'Description']:
        df[col] = clean_values(df[col], col)

columns_to_remove = ['OrdNumL1','OrdNumL2','OrdNumL3','OrdNumL4','M2M']
df.drop(columns=columns_to_remove, inplace=True)

# Convert specific columns to integer
df['TRDPARITY'] = df['TRDPARITY'].astype(int)/ 100
df['QTY'] = df['QTY'].astype(int)
df['ParityWas'] = df['ParityWas'].astype(int)/ 100


new_columns = {
    'Description': 'Portfolio',
    'ParityWas': 'ParityAsked',
}

df = df.rename(columns=new_columns)

#! Apply the alpha function to each row

df['alpha'] = df.apply(calculate_alpha, axis=1)

#! Alpha %

positive_alpha = round(df[df['alpha'] > 0]['alpha'].sum(),2)
negative_alpha = round(df[df['alpha'] < 0]['alpha'].sum(),2)
net_alpha = round(positive_alpha + negative_alpha,2)

# print(f'% positive_alpha : {positive_alpha}\n% negative_alpha : {negative_alpha}\n% net_alpha : {net_alpha}')

#! Alpha Events

positive_alpha_events = f'{round(((df['alpha'] >= 0).sum()/len(df))*100,2)} %'
negative_alpha_events = f'{round(((df['alpha'] < 0).sum()/len(df))*100,2)} %'

# print(f'% positive_alpha_events : {positive_alpha_events}\n% negative_alpha_events : {negative_alpha_events}')

#! Top 5 profitable/Lossers alphas

grouped_df = df.groupby('Portfolio', as_index=False)['alpha'].sum()
top_5_df = grouped_df.nlargest(5, 'alpha')
bottom_5_df = grouped_df.nsmallest(5, 'alpha')

#! Final dataframe 

top_alpha_portfolio = top_5_df['Portfolio'].to_list()
bottom_alpha_portfolio = bottom_5_df['Portfolio'].to_list()

data = {
        'dealer_id' : [98765],

        'positive_alpha_events' : [positive_alpha_events],
        'negative_alpha_events' : [negative_alpha_events],
        
        'positive_alpha' : [positive_alpha],
        'negative_alpha' : [negative_alpha],
        'net_alpha' : [net_alpha],

        't1' : [top_alpha_portfolio[0]],
        't2' : [top_alpha_portfolio[1]],
        't3' : [top_alpha_portfolio[2]],
        't4' : [top_alpha_portfolio[3]],
        't5' : [top_alpha_portfolio[4]],

        'b1' : [bottom_alpha_portfolio[0]],
        'b2' : [bottom_alpha_portfolio[1]],
        'b3' : [bottom_alpha_portfolio[2]],
        'b4' : [bottom_alpha_portfolio[3]],
        'b5' : [bottom_alpha_portfolio[4]]   
    }

df = pd.DataFrame(data)
df

,dealer_id,positive_alpha_events,negative_alpha_events,positive_alpha,negative_alpha,net_alpha,t1,t2,t3,t4,t5,b1,b2,b3,b4,b5
0,98765,60.86 %,39.14 %,13789.2,-6554.5,7234.7,NIFTY24523-20550-22400CE,MIDCPNIFTY24MAY-10800-11300CE,FINNIFTY24521-22500-21450PE,MIDCPNIFTY24MAY-11050-11300CE,NIFTY24MAY-22100-22400CE,NIFTY24523-22200-22350CE,MIDCPNIFTY24517-11300-11275PE,FINNIFTY24521-21300-21500CE,MIDCPNIFTY24517-11000-11250CE,MIDCPNIFTY24517-11325-11275PE
